### Voor- en nadelen van ORM
- `+` SQL syntax check door Python --> fouten ontdekt bij het ontwikkelen
- `+` Porteerbaar tussen databanksystemen
- `+` Integratie met Python-objecten
- `-` Extra syntax moet aangeleerd worden
- `-` Risico op trage code (vooral bij Optie 2: rechtstreeks gebruik van objecten) omdat men niet meer stilstaat bij gegenereerde SQL-commando's: 
  - SELECT *: onnodig veel kolommen ophalen
  - teveel "round-trips" naar de database, door bijv. in een lus updates uit te voeren i.p.v. in één update-statement

In [1]:
from sqlalchemy import create_engine  
from sqlalchemy.orm import Session

engine = create_engine('mssql+pyodbc://localhost\\SQL2025/xtreme?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
conn = engine.connect() 
session = Session(engine)

In [2]:
from sqlalchemy import String, select
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

from decimal import Decimal
from typing_extensions import Annotated
from sqlalchemy import Numeric
from sqlalchemy.orm import registry
num_8_2 = Annotated[Decimal, 8]

class Base(DeclarativeBase):
    registry = registry(
        type_annotation_map={
            num_8_2: Numeric(6, 2)
        }
    )

# Option 1: explicit Table objects
       
class Product(Base):
    __tablename__ = "Product"
    ProductID: Mapped[int] = mapped_column(primary_key=True)
    ProductName: Mapped[str]
    Color: Mapped[str]
    Sizes: Mapped[str]
    M_F: Mapped[str]
    Price: Mapped[float]
    ProductTypeID: Mapped[int]
    ProductClassID: Mapped[int]
    SupplierID: Mapped[int]
    ReorderLevel: Mapped[int]
    UnitsInStock: Mapped[int]

# Option 2: reflecting tables: generate Table objects automatically from database   

Base.metadata.reflect(engine)   # get metadata from database

class OrdersDetail(Base):  # each table is a subclass from the Base table
    __table__ = Base.metadata.tables['OrdersDetail']

class ProductType(Base):  
    __table__ = Base.metadata.tables['ProductType']
  
class Courier(Base):
    __table__ = Base.metadata.tables['Courier']




### Select

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [ ]:
stmt = select(Product.ProductID,Product.ProductName, (Product.UnitsInStock).label('Stock')) \
        .order_by(Product.UnitsInStock.desc()) \
        .limit(5)

products = session.execute(stmt)

**Complexere SELECT** 
   
```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [ ]:
from sqlalchemy import func

stmt = select(Product.ProductID,Product.ProductName,func.sum(OrdersDetail.Quantity * OrdersDetail.UnitPrice).label('Sales')) \
        .select_from(Product) \
        .join(OrdersDetail, Product.ProductID == OrdersDetail.ProductID) \
        .group_by(Product.ProductID, Product.ProductName) \
        .order_by(func.sum(OrdersDetail.Quantity * OrdersDetail.UnitPrice).desc()) \
        .limit(10)

rows = session.execute(stmt)

**UPDATE** \
Verhoog de prijzen van alle producten van het producttype x met y % (x en y zijn parameters). 

   ```sql
   update Product 
   set Price = Price * 1.10
   where ProductTypeID 
   in (select ProductTypeID 
   from Producttype where producttypename='Mountain')
   ``` 

In [ ]:
from sqlalchemy import update, bindparam
stmt = update(Product).values(Price=Product.Price * (1 + bindparam("pct"))).where(Product.ProductTypeID.in_(select(ProductType.ProductTypeID).where(ProductType.ProductTypeName == bindparam("type"))))
print(stmt) # stringify the statement
session.execute(stmt,{"pct":0.1, "type":"Mountain"}) 

conn.rollback()
conn.commit()



# Option 2: with ORM objects
pct = 0.1
type = 'Mountain'

stmt = select(Product).where(Product.ProductTypeID.in_(select(ProductType.ProductTypeID).where(ProductType.ProductTypeName == type))) 
print(stmt)  # stringify the statement
products = session.scalars(stmt)  # with scalars() we receive ORM entities directly

for product in products:
    product.Price = product.Price * (1 + pct)

conn.rollback()
conn.commit()

**INSERT**  
    ```sql
   insert into Courier (CourierID, CourierName,Website)
   values (11,'bpost','www.bpost.be')
   ```

In [ ]:
bpost = Courier(CourierID=11, CourierName='bpost', Website='www.bpost.be')
session.add(bpost)

conn.rollback()
conn.commit()

**DELETE**
   ```sql
   delete from Courier where CourierName = 'bpost'
    ```

In [ ]:
stmt = select(Courier).where(Courier.CourierName == 'bpost')
result = session.scalars(stmt).all()  # with scalars() we receive ORM entities directly,  with all() we get all results as a list
if len(result) > 0:  
    print('Courier bpost exists')
    bpost = result[0]   # get first courier with name bpost
    session.delete(bpost)
    print('Courier bpost deleted')
else:
    print('Courier bpost does not exist')

conn.rollback()
conn.commit()

In [ ]:
conn.close()